In [1]:
import numpy as np

In [77]:
def normer_forste_element(vektor):
    """
    Normaliserer en vektor ved å dele alle elementer på det første ikke-null elementet.
    
    Parametere:
        vektor (np.ndarray): En 1D Numpy-array som skal normaliseres.
    
    Returnerer:
        np.ndarray: En normalisert vektor der det første ikke-null elementet er 1.
    """
    if np.allclose(vektor, 0):
        return vektor
    # Finn indeksen til det første elementet i vektoren som ikke er null
    forste_ikke_null_indeks = np.argmax(vektor != 0)
    
    # Hent verdien til det første ikke-null elementet
    forste_ikke_null_element = vektor[forste_ikke_null_indeks]
    
    # Returner den normaliserte vektoren
    return vektor / forste_ikke_null_element

def gauss_jordan(matrise, epsilon=1e-8):
    """
    Utfører Gauss-Jordan eliminasjon på en gitt matrise.
    
    Parametere:
        matrise (np.ndarray): En 2D Numpy-array som representerer matrisen.
        epsilon (float): En liten verdi for å håndtere numerisk presisjon.
    
    Returnerer:
        np.ndarray: En rekkeredusert matrise i redusert trappeform.
    """
    # Sett svært små verdier til null for å unngå numeriske feil
    matrise[np.abs(matrise) < epsilon] = 0
    
    # Hvis matrisen kun består av nuller, returner den uendret
    if np.allclose(matrise, 0):
        return matrise
    
    # Hvis matrisen har én rad, normaliser den første radens første ikke-null element
    elif len(matrise) == 1:
        matrise[0] = normer_forste_element(matrise[0])
        return matrise
    
    # Beregn radenes summer for å normalisere matrisen (gjør tallene mer håndterbare)
    rad_maksimummer = np.max(np.abs(matrise), axis=1)
    rad_maksimummer[rad_maksimummer <= epsilon] = 1
    matrise = matrise / rad_maksimummer[:, None]  # Normaliser hver rad
    # Finn kolonner som inneholder ikke-null elementer
    ikke_null_kolonner = np.flatnonzero(np.any(matrise != 0, axis=0))
    
    # Finn indeksen til den første kolonnen med ikke-null elementer
    forste_ikke_null_kolonne_index = ikke_null_kolonner[np.argmax(np.max(np.abs(matrise[:, ikke_null_kolonner]), axis=0))]
    forste_ikke_null_kolonne = matrise[:, forste_ikke_null_kolonne_index]
    # Finn indeksen til raden med største verdi i den valgte kolonnen (pivot rad)
    pivot_rad_indeks = np.argmax(forste_ikke_null_kolonne)
    
    # Normaliser pivot-raden
    pivot_rad = normer_forste_element(matrise[pivot_rad_indeks])
    
    # Bytt plass på pivot-raden og den første raden
    matrise[pivot_rad_indeks] = matrise[0]
    matrise[0] = pivot_rad
    # Utfør eliminering for å gjøre alle elementene under pivoten null
    if matrise[0, forste_ikke_null_kolonne_index] > epsilon:
        matrise[1:] -= (matrise[1:, forste_ikke_null_kolonne_index] / matrise[0, forste_ikke_null_kolonne_index])[:, None] * matrise[0]

    # Kall Gauss-Jordan rekursivt på den nedre delmatrisen

    resterende_kolonner = np.concatenate((np.arange(0, forste_ikke_null_kolonne_index), np.arange(forste_ikke_null_kolonne_index + 1, matrise.shape[1])))

    matrise[1:, resterende_kolonner] = gauss_jordan(matrise[1:, resterende_kolonner])
    
    # Gjør den første raden null over pivot-posisjonene til de øvrige radene.
    matrise_pivot_posisjoner = pivot_posisjoner(matrise[1:])
    for rad_idx, col_idx in zip(*pivot_posisjoner(matrise[1:])):
        rad = matrise[1 + rad_idx]
        matrise[0] -= (matrise[0, col_idx] / rad[col_idx]) * rad 
    # for rad in matrise[1:]:
    #     if np.any(rad != 0):  # Hvis raden ikke er null
    #         # Finn indeksen til det første ikke-null elementet i raden
    #         forste_ikke_null_kolonne_index = np.argmax(rad != 0)
            
    #         # Trekk fra et multiplum av denne raden for å gjøre elementet over pivot null
    #         matrise[0] -= (matrise[0, forste_ikke_null_kolonne_index] / rad[forste_ikke_null_kolonne_index]) * rad
    
    # Returner den resulterende matrisen
    return matrise


In [123]:
def pivot_posisjoner(matrise):
    """
    Finner pivotposisjonene i en rekkeredusert matrise.
    """
    pivot_rader = []
    pivot_kolonner = []
    for rad_indeks, rad in enumerate(matrise):
        if np.any(rad != 0):
            første_ikke_null_kolonne = np.argmax(rad != 0)
            pivot_rader.append(rad_indeks)
            pivot_kolonner.append(første_ikke_null_kolonne)
    return pivot_rader, pivot_kolonner

def frie_parametre(matrise):
    """
    Finner bundne og frie parametere i en rekkeredusert matrise.
    """
    _, pivot_kolonner = pivot_posisjoner(matrise)
    alle_kolonner = set(range(matrise.shape[1]))
    return sorted(alle_kolonner.difference(pivot_kolonner))

def null_rom(matrise, epsilon=1e-8):
    """
    Finner en basis for nullrommet til en matrise.
    """
    nullrom_basis = []
    redusert_matrise = gauss_jordan(matrise, epsilon=epsilon)
    pivot_rader, pivot_kolonner = pivot_posisjoner(redusert_matrise)
    frie = frie_parametre(redusert_matrise)
    
    for fri in frie:
        vektor = np.zeros((matrise.shape[1], 1))
        vektor[fri] = 1
        for rad, kolonne in zip(pivot_rader, pivot_kolonner):
            vektor[kolonne] = -np.sum((redusert_matrise @ vektor)[rad]) / redusert_matrise[rad, kolonne]
        nullrom_basis.append(vektor)
    
    return nullrom_basis

def partikulaer_losning(koeffisientmatrise, høyreside=None, epsilon=1e-8):
    """
    Finner en partikulær løsning til ligningssystemet Ax = b.
    """
    if høyreside is None:
        høyreside = np.zeros(koeffisientmatrise.shape[1])
    if len(høyreside.shape) == 1:
        høyreside = høyreside[:, None]

    utvidet_null_rom = null_rom(np.hstack([koeffisientmatrise, høyreside]), epsilon=epsilon)
    utvidet_null_rom = [v for v in utvidet_null_rom if v[-1] > epsilon]
    assert len(utvidet_null_rom) > 0, "Det finnes ingen løsning til det lineære ligningssystemet"
    
    v = utvidet_null_rom[0]
    løsning = -v[: -1] / v[-1]
    
    assert np.allclose(koeffisientmatrise @ løsning[None, :], høyreside), "Det finnes ingen løsning til det lineære ligningssystemet"
    return løsning

In [124]:
# La oss bruke koden vår på denne matrisen

A = np.array([
    [3, 1, 3, 1, 4, 2],    
    [3, 1, 3, 1, 4, 1]
])

gauss_jordan(A)

array([[ 1.        ,  0.33333333,  1.        ,  0.33333333,  1.33333333,
         0.        ],
       [-0.        , -0.        , -0.        , -0.        ,  0.        ,
         1.        ]])

In [125]:
A = np.array([
    [2, 3],
    [4, 5]
])

b = np.array([5, 9])

In [126]:
x = partikulaer_losning(A, b)

In [127]:
gauss_jordan(A)

array([[0. , 1.5],
       [1. , 0. ]])

In [128]:
A @ x

array([[5.],
       [9.]])

For å vite hvor mange løsninger det finnes kan vi se på nullrommet

In [129]:
null_rom(A)

[]

Dette betyr at nullvektoren $0$ er eneste vektor $\vec v$ med $A \cdot \vec v = 0$ 

## Eksempel

Se på ligningssystemet
$$
\begin{array}{ccccccc}
    1x_0 &+& 2x_1& +& 3x_2& = &2 \\
    4x_0 &+& 5x_1& +& 6x_2& = &5 \\
    7x_0 &+& 8x_1& +& 9x_2& = &8 
\end{array}
$$


Gauss Jordan reduksjon blir

$$
\left[
\begin{array}{ccc|c}
    1 & 2 & 3 & 2 \\
    4 & 5 & 6 & 5 \\
    7 & 8 & 9 & 8
\end{array}
\right]
\sim
\left[
\begin{array}{ccc|c}
    1 & 2 & 3 & 2 \\
    0 & -3 & -6 & -3 \\
    0 & -6 & -12 & -6
\end{array}
\right]
\begin{array}{c}
    \\
    \mathrm{II} - 4\mathrm{I} \\
    \mathrm{III} - 7\mathrm{I} 
\end{array}
\sim
\left[
\begin{array}{ccc|c}
    1 & 2 & 3 & 2 \\
    0 & 1 & 2 & 1 \\
    0 & 0 & 0 & 0
\end{array}
\right]
\begin{array}{c}
    \\
    -\mathrm{II}/3\\
    \mathrm{III} - 2\mathrm{II} 
\end{array}
\sim
\left[
\begin{array}{ccc|c}
    1 & 0 & -1 & 0 \\
    0 & 1 & 2 & 1 \\
    0 & 0 & 0 & 0
\end{array}
\right]
\begin{array}{c}
    \mathrm{I} - 2\mathrm{II}\\ 
    \phantom{A}\\
    \phantom{A}
\end{array}
$$

Her leser vi av
$$
\begin{array}{cccccccc}
    x_0 &-& & & x_2& = &0 \\
     && x_1& +& 2x_2& = &1 \\
    &&&&0& = &0 
\end{array}
$$

Uansett verdien til $x_2$ har vi en løsning der $x_0 = x_2$ og $x_1 = 1 - 2x_2$.

Her skriver vi igjen inn koeffisientmatrisen og høyresiden:

In [ ]:
import numpy as np

In [130]:
A = np.array([
    [1., 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])

b = np.array([2, 5, 8])

In [131]:
x = partikulaer_losning(A, b)

In [132]:
x

array([[ 0.],
       [ 1.],
       [-0.]])

In [133]:
A @ x

array([[2.],
       [5.],
       [8.]])

In [93]:
null_rom_for_A = null_rom(A)

In [94]:
B = gauss_jordan(A)

In [95]:
B

array([[0. , 1.5, 3. ],
       [1. , 0.5, 0. ],
       [0. , 0. , 0. ]])

In [ ]:
Z = A[:, [2,1,0]]

In [ ]:
Z[:, np.argmax(np.max(np.abs(Z), axis=0))]

In [96]:
null_rom_for_A

[array([[ 1.],
        [-2.],
        [ 1.]])]

Vi ser at det er en vektor $\vec v$ forskjellig fra nullvektoren slik at $A \cdot \vec v = 0$

In [97]:
A

array([[1., 2., 3.],
       [4., 5., 6.],
       [7., 8., 9.]])

In [98]:
v = null_rom_for_A[0]
v

array([[ 1.],
       [-2.],
       [ 1.]])

In [99]:
A @ v

array([[0.0000000e+00],
       [8.8817842e-16],
       [0.0000000e+00]])

Siden $A \cdot \vec v = \vec 0$ har vi $A \cdot(\vec x + x_2 \vec v) = A \cdot \vec x + x_2 A \cdot \vec v = A \cdot \vec x
+ x_2 \vec 0 = A \cdot \vec x = \vec b$. 

Setter vi inn den $\vec x$ vi fant over med $A \cdot \vec x = \vec 0$ får vi

$$\vec x + x_2 \vec v =
\left[\begin{array}{c}
0 \\ 1 \\ 0 \end{array}\right]
+
x_2
\left[\begin{array}{c}
1 \\ -2 \\ 1 \end{array}\right]
=
\left[\begin{array}{c}
x_2 \\ 1 - 2x_2 \\ x_2 \end{array}\right]
$$
Dette er de samme løsningene som vi fant for hånd.

## Eksempel

Se på ligningssystemet
$$
\begin{array}{ccccccccccc}
    3x_0 &+& x_1& +& 3x_2& +& x_3 &+ &4x_4& = &2 \\
    3x_0 &+& x_1& +& 3x_2& +& x_3 &+ &4x_4& = &1
\end{array}
$$

In [134]:
A = np.array([
    [3, 1, 3, 1, 4],
    [3, 1, 3, 1, 4],
])

b = np.array([2, 1])

In [135]:
partikulaer_losning(A, b)

AssertionError: Det finnes ingen løsning til det lineære ligningssystemet

Dette ser farlig ut, men egentlig er det akkurat hva vi til ha. **Det finnes ingen løsning!**